In [1]:
# Lets first import some useful libraries for data analysis
import numpy as np
import pandas as pd
from pandas.plotting import _converter
_converter.register()
import matplotlib.pyplot as plt
import matplotlib.dates  as dates

# Create a dictionary of stock ticker symbols
ARCAMMI_Stocks={'AmericanExpress':'axp',
                'Boeing':'ba',
                'Chevron':'cvx',
                'DowDupont':'dwdp',
                'Disney':'dis',}

# Read in the historical data
historical_data = {}
for key, ticker in ARCAMMI_Stocks.items():
    historical_data[key] = pd.read_csv('MMI_DATA/'+ticker+'.txt',index_col=0)

In [2]:
def get_vwap(price,volume,d):
    """Return the VWAP anchored d days back"""
    vwap = np.zeros(len(price))
    for i in range(d,len(price)):
        lb = i - d #lower bound
        ub = i + 1 #upper bound
        vwap[i] = np.multiply(price[lb:ub],volume[lb:ub]).sum()
        vwap[i] /= np.sum(1.*volume[lb:ub])
    return vwap

def get_smap(price,d):
    """Return the SMAP anchored d days back"""
    smap = np.zeros(len(price))
    for i in range(d,len(price)):
        lb = i - d #lower bound
        ub = i + 1 #upper bound
        smap[i] = np.sum(price[lb:ub])/(d+1.)
    return smap

In [3]:
for n in [2,3]:
    lookback = n
    for company in historical_data:
        historical_data[company]['VWAP' + str(lookback)] = get_vwap(historical_data[company]['Close'],historical_data[company]['Volume'],lookback)
        historical_data[company]['SMAP' + str(lookback)] = get_smap(historical_data[company]['Close'],lookback)

c:\users\owner\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


In [4]:
historical_data['Disney'].tail(1)

High         Low        Open       Close      Volume  \
Date                                                                     
2019-03-28  111.269997  110.239998  110.599998  110.709999  10845900.0   

             Adj Close       VWAP2       SMAP2       VWAP3   SMAP3  
Date                                                                
2019-03-28  110.709999  110.340695  110.376666  109.519929  109.73

In [5]:
# Here define some functions to calculate our signals and values 

# if the price is greater than VWAP or SMAP, we want to go long (1), otherwise we want to be in a cash position (0)
def set_signal(metric, price, lookback):
    signal = np.zeros(len(price))
    for i in range(len(price)):
        if price[i] > metric[i] and i >= lookback and i < len(price) - 1:
            signal[i + 1] = 1
            
    return signal

# Now, we want to calculate the value of our portfolio
def set_value(signal,openprice):
    value = np.zeros(len(signal))
    value[0] = 1.
    for i in range(1,len(signal)):
        delta = (openprice[i] - openprice[i-1])/(openprice[i-1])
        value[i] = value[i-1]*(1 + signal[i-1]*delta)
        
    return value

In [6]:
# first get data in a new data frame for all stocks since 2010
start = '2010-01-01'
modern_data = {}


for company in historical_data:
    modern_data[company] = historical_data[company].copy().loc[start::]
for company in modern_data:  
    for n in [2,3]:
        lookback = n
    # note that the lookback time should be the same as the lookback used to calulate the SMAP and VWAP
        modern_data[company]['SMAP_Signal' + str(lookback)] = set_signal(modern_data[company]['SMAP' + str(lookback)], modern_data[company]['Close'], lookback)
        modern_data[company]['VWAP_Signal' + str(lookback)] = set_signal(modern_data[company]['VWAP' + str(lookback)], modern_data[company]['Close'], lookback)
        modern_data[company]['SMAP_Value' + str(lookback)]  = set_value (modern_data[company]['SMAP_Signal' + str(lookback)], modern_data[company]['Open'])
        modern_data[company]['VWAP_Value' + str(lookback)]  = set_value (modern_data[company]['VWAP_Signal' + str(lookback)], modern_data[company]['Open'])

        # also calculate the benchmark signal and value
    bm_signal = np.ones(len(modern_data[company]['Open']))
    bm_signal[0:15] = 0 # we don't want a headstart on the benchmark...
    modern_data[company]['BM_Signal'] = bm_signal
    modern_data[company]['BM_Value']  = set_value (modern_data[company]['BM_Signal'], modern_data[company]['Open'])
    
# and lets go ahead and take a look at Disney
modern_data['Disney'].tail(1)

High         Low        Open       Close      Volume  \
Date                                                                     
2019-03-28  111.269997  110.239998  110.599998  110.709999  10845900.0   

             Adj Close       VWAP2       SMAP2       VWAP3   SMAP3  \
Date                                                                 
2019-03-28  110.709999  110.340695  110.376666  109.519929  109.73   

            SMAP_Signal2  VWAP_Signal2  SMAP_Value2  VWAP_Value2  \
Date                                                               
2019-03-28           1.0           1.0     1.802886     1.930006   

            SMAP_Signal3  VWAP_Signal3  SMAP_Value3  VWAP_Value3  BM_Signal  \
Date                                                                          
2019-03-28           1.0           1.0     2.247437     2.011622        1.0   

            BM_Value  
Date                  
2019-03-28  3.711409

In [14]:
# First, lets get the relevant data
smaps2 = []
vwaps2 = []
smaps3 = []
vwaps3 = []
bms    = []

smaps     = [smaps2, smaps3]
vwaps     = [vwaps2, vwaps3]

for company in modern_data:
    for n in [2,3]:
        for index, smap in enumerate(smaps):
            smap.append(modern_data[company].iloc[-1]['SMAP_Value' + str(n)])
            vwaps[index].append(modern_data[company].iloc[-1]['VWAP_Value' + str(n)])
    bms.append(modern_data[company].iloc[-1]['BM_Value'])

smaps2 = np.array(smaps)
vwaps2 = np.array(vwaps)
smaps3 = np.array(smaps)
vwaps3 = np.array(vwaps)
bms    = np.array(bms)

smaps     = [smaps2, smaps3]
vwaps     = [vwaps2, vwaps3]
lookbacks = [2,3]

In [20]:
from scipy.stats import chisquare

def calc(vwap, smap):
    delta     = vwap - smap
    mean      = delta.mean()
    std       = delta.std()
    chi       = chisquare(vwap, smap)
    pvalue    = chi[1]
    
    return delta, mean, std, pvalue

In [19]:
for index, smap in enumerate(smaps):
    calc(vwaps[index], smap)